In [1]:
import pandas as pd
import numpy as np

import os
import json
import random
import openai
import google.generativeai as genai
import anthropic
from openai import OpenAI
from datasets import load_dataset
from tqdm import tqdm
from dotenv import load_dotenv

In [3]:
# This line reads the .env file and loads the variables into the environment
load_dotenv() 

#get the key you loaded from the .env file
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    client = OpenAI(api_key=api_key)
    print("OpenAI client initialized successfully.")
else:
    raise ValueError("OpenAI API key not found. Make sure it's set in your .env file.")

models = client.models.list()
for model in models.data:
    print(model.id)

OpenAI client initialized successfully.
gpt-4-0613
gpt-4
gpt-3.5-turbo
gpt-4o-audio-preview-2025-06-03
gpt-4.1-nano-2025-04-14
gpt-4.1-nano
gpt-image-1
gpt-4o-realtime-preview-2025-06-03
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-4-1106-preview
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-4-0125-preview
gpt-4-turbo-preview
gpt-3.5-turbo-0125
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
chatgpt-4o-latest
o1-preview-2024-09-12
o1-preview
o1-mini-2024-09-12
o1-mini
gpt-4o-realtime-preview-2024-10-01
gpt-4o-audio-preview-2024-10-01
gpt-4o-audio-preview
gpt-4o-realtime-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-realtime-preview-2024-12-17
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-realtime-preview
g

In [4]:
# Load the source dataset once
gsm8k_train = load_dataset("gsm8k", "main")['train']
print("Dataset loaded.")


# Define the taxonomy of unanswerability we will use
UNANSWERABILITY_TAXONOMY = {
    "insufficient_information": "Make the problem unanswerable by removing a single, critical piece of numerical information. For example, if a problem mentions the cost of apples and oranges, remove the cost of apples.",
    "contradictory_information": "Make the problem unanswerable by adding a piece of information that directly contradicts another statement in the problem. For example, if a problem states there are 10 apples, add a sentence stating there are 12 apples.",
    # "ambiguous_question": "Make the problem unanswerable by making the final question ambiguous. The numbers and facts should remain, but the question itself should be interpretable in two or more ways, making a single answer impossible.",
    # "no_solution_possible": "Make the problem unanswerable by changing a number or condition so the premise becomes mathematically impossible. For example, a baker sells 5 cakes for $20 total, and makes a profit of $25.",
}

MODIFICATION_PAIRS = list(UNANSWERABILITY_TAXONOMY.items())

Dataset loaded.


In [5]:
# The core function that calls the LLM
def make_problem_unanswerable(problem_text, modification_type, modification_instruction, model):
    system_prompt = "You are an expert in curriculum design and mathematical pedagogy. Your task is to subtly modify a solvable math problem to make it unanswerable, for the purpose of testing a student's critical thinking."

    user_prompt = f"""
    Please rewrite the following math problem.

    **Original Problem:**
    "{problem_text}"

    **Modification Type:**
    {modification_type}

    **Instruction:**
    {modification_instruction}

    **Your Task:**
    1.  Rewrite the problem according to the instruction.
    2.  Make the *minimal necessary change*. The problem should still look like a plausible, well-formed math problem.
    3.  Do NOT use placeholders like '[missing information]' or '[contradiction]'. The change should be subtle.
    4.  Output a JSON object with three keys:
        - "unanswerable_problem": The full text of the newly generated unanswerable problem.
        - "change_summary": A brief, one-sentence description of what you changed.
        - "reasoning": A clear explanation of why the new problem is unanswerable, directly referencing the modification type.

    Example JSON output format:
    {{
      "unanswerable_problem": "A bakery sells chocolate cakes for $18. On a certain day, it sold 10 cakes in total. How many chocolate cakes did it sell?",
      "change_summary": "I removed the price of vanilla cakes and the total revenue.",
      "reasoning": "This problem is now unanswerable due to insufficient_information. It is impossible to determine the number of each type of cake sold without knowing either the price of the other cake or the total revenue."
    }}
    """

    try:
        kwargs = dict(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            response_format={"type": "json_object"}
        )
        if not model.lower().startswith("o"):
            kwargs["temperature"] = 0.5  # Lower temperature for more predictable, instruction-following behavior

        response = client.chat.completions.create(**kwargs)
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        print(f"An API error occurred: {e}")
        return None

def run_baby_step_experiment(model, save_folder, num_samples=6, random_seed=42):
    """
    Run a baby-step experiment to generate unanswerable math problems.
    
    Args:
        model (str): The OpenAI model to use for generation.
        savepath (str): Path to save the generated dataset.
        num_samples (int): Number of samples to generate.
    """
    # Get a random subset of the data to work with
    random.seed(random_seed)
    indices = random.sample(range(len(gsm8k_train)), num_samples)

    print(f"Starting generation of {num_samples} samples with model {model}...")

    savepath = f'{save_folder}/outputs_{model}.jsonl'
    with open(savepath, 'w') as f:
        # Using tqdm for a progress bar, which works great in notebooks
        for i, index in enumerate(tqdm(indices)):
            original_problem = gsm8k_train[index]['question']
            if i % 2 == 0:
                mod_type_key, mod_instruction = MODIFICATION_PAIRS[0]
            else:
                mod_type_key, mod_instruction = MODIFICATION_PAIRS[1]

            generated_data = make_problem_unanswerable(original_problem, mod_type_key, mod_instruction, model)
            
            if generated_data:
                final_record = {
                    "original_problem": original_problem,
                    "unanswerable_problem": generated_data.get("unanswerable_problem"),
                    "modification_type": mod_type_key,
                    "change_summary": generated_data.get("change_summary"),
                    "reasoning": generated_data.get("reasoning"),
                }
                f.write(json.dumps(final_record) + "\n")

    print(f"\nGeneration complete.")

In [6]:
models_to_test = ["gpt-4o",
                  "gpt-4.1-mini",
                  "gpt-4.1",
                  "o3-mini",
                  "o3-mini-2025-01-31"]

In [7]:
# for model in models_to_test:
#     run_baby_step_experiment(model, 'bse-outputs_june-13', num_samples=10, random_seed=42)
#     print()

In [8]:
def make_output_dfs_dict(models, output_folder):
    output_dict = {}
    for model in models:
        data = []
        with open(f'{output_folder}/outputs_{model}.jsonl', 'r') as f:
            for line in f:
                data.append(json.loads(line))
        output_dict[model] = data

    output_dfs = {model: pd.DataFrame(data) for model, data in output_dict.items()}

    # Concatenate all model outputs, adding a 'model' column
    all_rows = []
    for model, df in output_dfs.items():
        temp = df.copy()
        temp['model'] = model
        all_rows.append(temp)
    combined_df = pd.concat(all_rows, ignore_index=True)

    # Group by 'original_problem' and build the desired dictionary
    problem_dict = {}
    for problem, group in combined_df.groupby('original_problem'):
        # Set model as index, select only the relevant columns
        sub_df = group.set_index('model')[['unanswerable_problem', 'modification_type', 'change_summary', 'reasoning']]
        problem_dict[problem] = sub_df

    return problem_dict

def display_problem_dict(problem_dict):
    pd.set_option('display.max_colwidth', None)
    original_problems = list(problem_dict.keys())

    for i, (problem, sub_df) in enumerate(problem_dict.items()):
        print(f"Problem {i+1}: {problem}")
        display(sub_df)

In [9]:
problem_dict = make_output_dfs_dict(models_to_test, 'bse-outputs_june-13')
display_problem_dict(problem_dict)

Problem 1: Aaron pays his actuary membership fees each year. The membership fee increases yearly by $10. If he pays $80 in the first year, how much does his membership cost, in dollars, in the sixth year?


,unanswerable_problem,modification_type,change_summary,reasoning
model,,,,
gpt-4o,"Aaron pays his actuary membership fees each year. The membership fee decreases yearly by $10. If he pays $80 in the first year, how much does his membership cost, in dollars, in the sixth year?",contradictory_information,I changed the statement about the membership fee increasing by $10 to decreasing by $10.,"This problem is now unanswerable due to contradictory_information. One statement says the fee increases by $10 each year, while another says it decreases by $10 each year, making it impossible to determine the cost in the sixth year."
gpt-4.1-mini,"Aaron pays his actuary membership fees each year. The membership fee increases yearly by $10. If he pays $80 in the first year, his membership fee remains $75 in the sixth year. How much does his membership cost, in dollars, in the sixth year?",contradictory_information,"I added a sentence stating the membership fee remains $75 in the sixth year, contradicting the stated yearly increase.","This problem is now unanswerable due to contradictory_information. The statement that the fee increases yearly by $10 conflicts directly with the claim that the fee remains $75 in the sixth year, making it impossible to determine the true membership cost in that year."
gpt-4.1,"Aaron pays his actuary membership fees each year. The membership fee increases yearly by $10. If he pays $80 in the first year, and the fee remains the same each year, how much does his membership cost, in dollars, in the sixth year?",contradictory_information,"I added the statement that the fee remains the same each year, directly contradicting the earlier statement that the fee increases yearly by $10.",This problem is unanswerable due to contradictory_information: it is impossible to determine the sixth year's fee when the problem states both that the fee increases by $10 each year and also that it remains the same each year.
o3-mini,"Aaron pays his actuary membership fees each year. The membership fee increases yearly by $10. However, the fee remains fixed at $80 every year. If he pays $80 in the first year, how much does his membership cost, in dollars, in the sixth year?",contradictory_information,"I added a sentence stating that the fee remains fixed at $80 every year, which contradicts the statement that it increases by $10 annually.","The problem is unanswerable due to contradictory_information: the problem states that the fee increases each year by $10, but also that it remains constant at $80, creating a direct contradiction in the conditions."
o3-mini-2025-01-31,"Aaron pays his actuary membership fees each year. The membership fee increases yearly by $10. However, his fee is fixed at $80 every year. If he pays $80 in the first year, how much does his membership cost, in dollars, in the sixth year?",contradictory_information,"I added a sentence stating that the fee is fixed at $80 every year, which contradicts the assertion that the fee increases by $10 annually.","The problem is unanswerable due to contradictory_information: one statement claims the fee increases yearly by $10 while another states it always remains $80, creating a direct conflict about the fee's behavior."


Problem 2: Betty picked 16 strawberries. Matthew picked 20 more strawberries than Betty and twice as many as Natalie. They used their strawberries to make jam. One jar of jam used 7 strawberries and they sold each jar at $4. How much money were they able to make from the strawberries they picked?


,unanswerable_problem,modification_type,change_summary,reasoning
model,,,,
gpt-4o,"Betty picked 16 strawberries. Matthew picked 20 more strawberries than Betty and twice as many as Natalie. However, Natalie picked 10 more strawberries than Matthew. They used their strawberries to make jam. One jar of jam used 7 strawberries and they sold each jar at $4. How much money were they able to make from the strawberries they picked?",contradictory_information,I added a statement that Natalie picked 10 more strawberries than Matthew.,"The problem is now unanswerable because of contradictory_information. Matthew's strawberry count is defined in relation to both Betty's and Natalie's counts, but the added statement about Natalie picking more than Matthew contradicts the initial relationship, making it impossible to determine the actual number of strawberries each person picked."
gpt-4.1-mini,"Betty picked 16 strawberries. Matthew picked 20 more strawberries than Betty and twice as many as Natalie. However, Natalie picked 25 strawberries. They used their strawberries to make jam. One jar of jam used 7 strawberries and they sold each jar at $4. How much money were they able to make from the strawberries they picked?",contradictory_information,"I added that Natalie picked 25 strawberries, contradicting the relationship between Matthew's and Natalie's strawberry counts.","This problem is now unanswerable due to contradictory_information. The original problem states Matthew picked twice as many strawberries as Natalie, but the added statement that Natalie picked 25 strawberries conflicts with the earlier information about Matthew's count, making it impossible to determine the exact number of strawberries each person picked."
gpt-4.1,"Betty picked 16 strawberries. Matthew picked 20 more strawberries than Betty and twice as many as Natalie. However, Betty actually picked 24 strawberries. They used their strawberries to make jam. One jar of jam used 7 strawberries and they sold each jar at $4. How much money were they able to make from the strawberries they picked?",contradictory_information,"I added a sentence stating that Betty actually picked 24 strawberries, directly contradicting the earlier statement that she picked 16.","This problem is now unanswerable due to contradictory_information. Since Betty is said to have picked both 16 and 24 strawberries, it is impossible to determine the correct number of strawberries she picked, making the rest of the calculations indeterminate."
o3-mini,"Betty picked 16 strawberries. Matthew picked 20 more strawberries than Betty and twice as many as Natalie. However, a second report indicates that Betty only picked 14 strawberries. They used their strawberries to make jam. One jar of jam used 7 strawberries and they sold each jar at $4. How much money were they able to make from the strawberries they picked?",contradictory_information,I inserted a sentence that contradicts Betty's original count by stating she picked 14 strawberries instead of 16.,"The problem is unanswerable because the contradictory information about Betty's strawberry count creates ambiguity, making it impossible to determine the correct number of strawberries for further calculations."
o3-mini-2025-01-31,"Betty picked 16 strawberries. However, another report states that Betty picked 18 strawberries. Matthew picked 20 more strawberries than Betty and twice as many as Natalie. They used their strawberries to make jam. One jar of jam used 7 strawberries and they sold each jar at $4. How much money were they able to make from the strawberries they picked?",contradictory_information,I added an extra sentence that contradicts the originally stated number of strawberries Betty picked.,"The problem is now unanswerable because it provides two conflicting numbers for the strawberries Betty picked, making it impossible to determine the correct quantities needed to solve the problem."


Problem 3: For every 12 cans you recycle, you receive $0.50, and for every 5 kilograms of newspapers, you receive $1.50. If your family collected 144 cans and 20 kilograms of newspapers, how much money would you receive?


...

,unanswerable_problem,modification_type,change_summary,reasoning
model,,,,
gpt-4o,"Joseph invested $1000 into a hedge fund. The fund promised a yearly interest rate. If he deposited an additional $100 every month into the account to add to his initial investment of $1000, how much money will he have in the fund after two years?",insufficient_information,I removed the specific yearly interest rate of 10%.,"This problem is now unanswerable due to insufficient_information. Without the specific yearly interest rate, it is impossible to calculate the total amount of money Joseph will have after two years, as the interest rate is crucial for calculating the growth of his investment."
gpt-4.1-mini,"Joseph invested $1000 into a hedge fund. The fund promised a yearly interest rate. If he deposited an additional $100 every month into the account to add to his initial investment of $1000, how much money will he have in the fund after two years?",insufficient_information,I removed the numerical value of the yearly interest rate.,"This problem is now unanswerable due to insufficient_information. Without a specific interest rate, it is impossible to calculate the growth of the investment over two years."
gpt-4.1,"Joseph invested an amount of money into a hedge fund. The fund promised a yearly interest rate of 10%. If he deposited an additional $100 every month into the account to add to his initial investment, how much money will he have in the fund after two years?",insufficient_information,I removed the specific value of Joseph's initial investment ($1000).,"This problem is now unanswerable due to insufficient_information. Without knowing the amount of the initial investment, it is impossible to calculate the total value in the fund after two years."
o3-mini,"Joseph invested $1000 into a hedge fund. The fund promised a yearly interest rate. If he deposited an additional $100 every month into the account to add to his initial investment of $1000, how much money will he have in the fund after two years?",insufficient_information,I removed the numerical value of the yearly interest rate.,"This problem is now unanswerable due to insufficient information because the critical interest rate value is missing, making it impossible to compute the growth of the investment."
o3-mini-2025-01-31,"Joseph invested $1000 into a hedge fund. The fund promised a yearly interest rate. If he deposited an additional $100 every month into the account to add to his initial investment of $1000, how much money will he have in the fund after two years?",insufficient_information,I removed the specific numerical value for the yearly interest rate.,"The problem is unanswerable due to insufficient information because without knowing the exact yearly interest rate, it's impossible to calculate the final amount in the fund."


Problem 9: The Rotary Club is holding its annual fundraising Omelet Breakfast, with tickets sold in advance. The tickets come in different price levels, for young children, older children, adults, and seniors. This year they sold 53 small children tickets, 35 older children tickets, 75 adult tickets, and 37 senior tickets. To figure out how many eggs they need to buy, the club estimates that small children can eat a half omelet, older children can eat a whole omelet, adults will eat two omelets, and seniors will eat one and a half omelets. Just to be on the safe side, they get enough eggs to make 25 extra omelets. If they use 2 eggs for each omelet, how many eggs will they need to buy?


,unanswerable_problem,modification_type,change_summary,reasoning
model,,,,
gpt-4o,"The Rotary Club is holding its annual fundraising Omelet Breakfast, with tickets sold in advance. The tickets come in different price levels, for young children, older children, adults, and seniors. This year they sold 53 small children tickets, 35 older children tickets, 75 adult tickets, and 37 senior tickets. To figure out how many eggs they need to buy, the club estimates that small children can eat a half omelet, older children can eat a whole omelet, adults will eat two omelets, and seniors will eat one and a half omelets. Just to be on the safe side, they get enough eggs to make 25 extra omelets. If they use 2 eggs for each omelet, how many eggs will they need to buy? Additionally, assume that adults actually eat only one omelet.",contradictory_information,"I added a statement that adults eat only one omelet, which contradicts the earlier statement that adults eat two omelets.","The problem is unanswerable because it contains contradictory information about how many omelets adults eat, making it impossible to calculate the total number of eggs needed accurately."
gpt-4.1-mini,"The Rotary Club is holding its annual fundraising Omelet Breakfast, with tickets sold in advance. The tickets come in different price levels, for young children, older children, adults, and seniors. This year they sold 53 small children tickets, 35 older children tickets, 75 adult tickets, and 37 senior tickets. To figure out how many eggs they need to buy, the club estimates that small children can eat a half omelet, older children can eat a whole omelet, adults will eat two omelets, and seniors will eat one and a half omelets. However, they also estimate that adults will eat only one omelet each this year. Just to be on the safe side, they get enough eggs to make 25 extra omelets. If they use 2 eggs for each omelet, how many eggs will they need to buy?",contradictory_information,"I added a contradictory estimate for the number of omelets adults will eat, stating adults will eat only one omelet, conflicting with the earlier estimate of two omelets each.","This problem is unanswerable due to contradictory_information. The problem provides two conflicting estimates for how many omelets adults will eat (two omelets versus one omelet), making it impossible to determine the correct total number of omelets needed."
gpt-4.1,"The Rotary Club is holding its annual fundraising Omelet Breakfast, with tickets sold in advance. The tickets come in different price levels, for young children, older children, adults, and seniors. This year they sold 53 small children tickets, 35 older children tickets, 75 adult tickets, and 37 senior tickets. To figure out how many eggs they need to buy, the club estimates that small children can eat a half omelet, older children can eat a whole omelet, adults will eat two omelets, and seniors will eat one and a half omelets. Just to be on the safe side, they get enough eggs to make 25 extra omelets. If they use 2 eggs for each omelet, how many eggs will they need to buy? (Note: Each omelet requires 3 eggs.)",contradictory_information,"I added a note at the end stating that each omelet requires 3 eggs, directly contradicting the earlier statement that each omelet uses 2 eggs.","This problem is now unanswerable because it contains contradictory_information: it gives two different values for the number of eggs per omelet (2 eggs and 3 eggs), making it impossible to determine which value should be used in the calculation."
o3-mini,"The Rotary Club is holding its annual fundraising Omelet Breakfast, with tickets sold in advance. The tickets come in different price levels, for young children, older children, adults, and seniors. This year they sold 53 small children tickets, 35 older children tickets, 75 adult tickets, and 37 senior tickets. However, a previous report stated that 40 senior tickets were sold. To figure out how many eggs they need to buy,

Problem 10: The price of buying a wooden toy at the new Craftee And Best store is $20, and the cost of buying a hat is $10. If Kendra went to the shop with a $100 bill and bought two wooden toys and three hats, calculate the change she received.


,unanswerable_problem,modification_type,change_summary,reasoning
model,,,,
gpt-4o,"The price of buying a wooden toy at the new Craftee And Best store is $20, and the cost of buying a hat is $10. If Kendra went to the shop with a $100 bill and bought two wooden toys and three hats, calculate the change she received. However, the store also has a policy that each wooden toy costs $15 on weekends.",contradictory_information,"I added a statement that wooden toys cost $15 on weekends, contradicting the initial price of $20.","This problem is now unanswerable due to contradictory_information. The price of the wooden toys is given as both $20 and $15, making it impossible to determine the correct total cost and, consequently, the change Kendra should receive."
gpt-4.1-mini,"The price of buying a wooden toy at the new Craftee And Best store is $20, and the cost of buying a hat is $10. If Kendra went to the shop with a $100 bill and bought two wooden toys and three hats, calculate the change she received. However, the total cost of the two wooden toys and three hats was $150.",contradictory_information,"I added a sentence stating the total cost of the items was $150, contradicting the earlier prices and quantities.","This problem is now unanswerable due to contradictory_information. The stated total cost ($150) conflicts with the product of the given prices and quantities (2 toys × $20 + 3 hats × $10 = $70), making it impossible to determine the correct change."
gpt-4.1,"The price of buying a wooden toy at the new Craftee And Best store is $20, and the cost of buying a hat is $10. If Kendra went to the shop with a $100 bill and bought two wooden toys and three hats, calculate the change she received. (Note: At Craftee And Best, all wooden toys are priced at $15 each.)",contradictory_information,"I added a sentence stating that all wooden toys are priced at $15 each, directly contradicting the earlier statement that a wooden toy costs $20.","This problem is now unanswerable due to contradictory_information, as there are two different prices given for wooden toys, making it impossible to determine which should be used to calculate the total cost."
o3-mini,"The price of buying a wooden toy at the new Craftee And Best store is $20, and the cost of buying a hat is $10. However, it is also stated that the price of a wooden toy is $25. If Kendra went to the shop with a $100 bill and bought two wooden toys and three hats, calculate the change she received.",contradictory_information,Added a contradictory statement about the wooden toy's price.,The problem is unanswerable because it provides conflicting information about the wooden toy's price—first stating it is $20 and later that it is $25—making it impossible to determine the correct cost for the calculation.
o3-mini-2025-01-31,"The price of buying a wooden toy at the new Craftee And Best store is $20, and the cost of buying a hat is $10. However, due to a special deal running that day, wooden toys are being sold for $25. If Kendra went to the shop with a $100 bill and bought two wooden toys and three hats, calculate the change she received.",contradictory_information,I introduced a contradictory statement about the wooden toy's cost.,The problem is unanswerable because it provides two conflicting prices for wooden toys—$20 and $25—making it impossible to determine the correct total cost and therefore the correct change.


In [10]:
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

models = genai.list_models()
print("Gemini models:")
for m in models:
    print(m.name)

Gemini models:
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/g

In [15]:
anthropic_client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

models = anthropic_client.models.list()
print("Anthropic Claude models:")
for m in models:
    print(m.id)

Anthropic Claude models:
claude-opus-4-20250514
claude-sonnet-4-20250514
claude-3-7-sonnet-20250219
claude-3-5-sonnet-20241022
claude-3-5-haiku-20241022
claude-3-5-sonnet-20240620
claude-3-haiku-20240307
claude-3-opus-20240229


In [18]:
def make_problem_unanswerable(problem_text, modification_type, modification_instruction, model):
    system_prompt = "You are an expert in curriculum design and mathematical pedagogy. Your task is to subtly modify a solvable math problem to make it unanswerable, for the purpose of testing a student's critical thinking."

    user_prompt = f"""
    Please rewrite the following math problem.

    **Original Problem:**
    "{problem_text}"

    **Modification Type:**
    {modification_type}

    **Instruction:**
    {modification_instruction}

    **Your Task:**
    1.  Rewrite the problem according to the instruction.
    2.  Make the *minimal necessary change*. The problem should still look like a plausible, well-formed math problem.
    3.  Do NOT use placeholders like '[missing information]' or '[contradiction]'. The change should be subtle.
    4.  Output a JSON object with three keys:
        - "unanswerable_problem": The full text of the newly generated unanswerable problem.
        - "change_summary": A brief, one-sentence description of what you changed.
        - "reasoning": A clear explanation of why the new problem is unanswerable, directly referencing the modification type.

    Example JSON output format:
    {{
      "unanswerable_problem": "A bakery sells chocolate cakes for $18. On a certain day, it sold 10 cakes in total. How many chocolate cakes did it sell?",
      "change_summary": "I removed the price of vanilla cakes and the total revenue.",
      "reasoning": "This problem is now unanswerable due to insufficient_information. It is impossible to determine the number of each type of cake sold without knowing either the price of the other cake or the total revenue."
    }}
    """

    try:
        if model.startswith("g:"):
            # Google Gemini
            gemini_model = model[2:]
            gemini = genai.GenerativeModel(gemini_model)
            response = gemini.generate_content(
                [system_prompt, user_prompt],
                generation_config={"response_mime_type": "application/json"}
            )
            return json.loads(response.text)
        elif model.startswith("a:"):
            # Anthropic Claude
            claude_model = model[2:]
            response = anthropic_client.messages.create(
                model=claude_model,
                max_tokens=512,
                system=system_prompt,
                messages=[{"role": "user", "content": user_prompt}]
            )
            return json.loads(response.content[0].text)
        elif model.startswith("o:"):
            # OpenAI
            openai_model = model[2:]
            kwargs = dict(
                model=openai_model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                response_format={"type": "json_object"}
            )
            if not openai_model.startswith("o"):
                kwargs["temperature"] = 0.5
            response = client.chat.completions.create(**kwargs)
            return json.loads(response.choices[0].message.content)
    except Exception as e:
        print(f"An API error occurred: {e}")
        return None
    

def run_baby_step_experiment(model, save_folder, num_samples=6, random_seed=42):
    """
    Run a baby-step experiment to generate unanswerable math problems.
    
    Args:
        model (str): The OpenAI model to use for generation.
        savepath (str): Path to save the generated dataset.
        num_samples (int): Number of samples to generate.
    """
    # Get a random subset of the data to work with
    random.seed(random_seed)
    indices = random.sample(range(len(gsm8k_train)), num_samples)

    print(f"Starting generation of {num_samples} samples with model {model}...")

    savepath = f'{save_folder}/outputs_{model}.jsonl'
    with open(savepath, 'w') as f:
        # Using tqdm for a progress bar, which works great in notebooks
        for i, index in enumerate(tqdm(indices)):
            original_problem = gsm8k_train[index]['question']
            if i % 2 == 0:
                mod_type_key, mod_instruction = MODIFICATION_PAIRS[0]
            else:
                mod_type_key, mod_instruction = MODIFICATION_PAIRS[1]

            generated_data = make_problem_unanswerable(original_problem, mod_type_key, mod_instruction, model)
            
            if generated_data:
                final_record = {
                    "original_problem": original_problem,
                    "unanswerable_problem": generated_data.get("unanswerable_problem"),
                    "modification_type": mod_type_key,
                    "change_summary": generated_data.get("change_summary"),
                    "reasoning": generated_data.get("reasoning"),
                }
                f.write(json.dumps(final_record) + "\n")

    print(f"\nGeneration complete.")

In [19]:
models_to_test = [
    "g:gemini-2.5-flash-preview-05-20",
    "a:claude-sonnet-4-20250514"
    ]

In [20]:
for model in models_to_test:
    run_baby_step_experiment(model, 'bse-outputs_june-13', num_samples=10, random_seed=42)
    print()

Starting generation of 10 samples with model g:gemini-2.5-flash-preview-05-20...


100%|██████████| 10/10 [01:19<00:00,  7.93s/it]



Generation complete.

Starting generation of 10 samples with model a:claude-sonnet-4-20250514...


 20%|██        | 2/10 [00:12<00:48,  6.11s/it]

An API error occurred: Expecting value: line 1 column 1 (char 0)


 70%|███████   | 7/10 [00:40<00:16,  5.61s/it]

An API error occurred: Expecting value: line 1 column 1 (char 0)


100%|██████████| 10/10 [01:00<00:00,  6.07s/it]


Generation complete.

